Difficulty Calculation:

The difficulty 1 target (from 0x1d00ffff) represents the easiest acceptable target.

The current  example target (from 0x1e0ffff0) is typically a lower number (i.e. a harder target) if the network difficulty has increased.

Dividing the difficulty 1 target by the current target gives you the difficulty factor. If the current target is lower, the difficulty factor will be greater than 1, meaning it's harder to find a valid hash.

In [5]:
def bits_to_target(bits):
    """
    Convert the compact representation of the target (bits) to the full target value.
    
    The 'bits' value is a 32-bit unsigned integer:
      - The most significant byte is the exponent.
      - The remaining 3 bytes are the coefficient.
    
    The target is computed as:
         target = coefficient * 2^(8*(exponent - 3))
    """
    exponent = bits >> 24             # Extract the exponent (top 8 bits)
    coefficient = bits & 0xffffff     # Extract the coefficient (lower 24 bits)
    target = coefficient * (1 << (8 * (exponent - 3)))
    return target

# Example bits values:
difficulty_1_bits = 0x1d00ffff   # Bitcoin's difficulty 1 bits
current_bits = 0x1e0ffff0        # Our example bits value

# Compute targets:
difficulty_1_target = bits_to_target(difficulty_1_bits)
current_target = bits_to_target(current_bits)

print("Difficulty 1 bits:", hex(difficulty_1_bits))
print("Difficulty 1 target:", hex(difficulty_1_target))
print("\nCurrent bits:", hex(current_bits))
print("Current target:", hex(current_target))


Difficulty 1 bits: 0x1d00ffff
Difficulty 1 target: 0xffff0000000000000000000000000000000000000000000000000000

Current bits: 0x1e0ffff0
Current target: 0xffff0000000000000000000000000000000000000000000000000000000


In [8]:
# Compute the difficulty factor
difficulty_factor = difficulty_1_target / current_target
print("\nComputed difficulty factor:", difficulty_factor)
print("Difficulty factor in percentage:", difficulty_factor * 100)


Computed difficulty factor: 0.000244140625
Difficulty factor in percentage: 0.0244140625


A computed difficulty factor of 0.000244140625 (which is 1/4096) means that the current target is 4096 times higher than the baseline (difficulty 1) target. In mining terms, a higher target makes it easier to find a hash below that target—so this result suggests that mining is much easier compared to the baseline difficulty.


In other words, if the baseline difficulty_1 target T1, then the current target is T1/0.000244140625=T1×4096.
Since a valid block hash must be less than the target, having a target that is 4096 times higher means there are many more acceptable hashes available, which greatly reduces the effort needed to mine a block.


Lets see a hard example

In [9]:
# Assume we use Bitcoin's difficulty 1 bits value.
difficulty_1_bits = 0x1d00ffff
difficulty_1_target = bits_to_target(difficulty_1_bits)

# Case 1: Easy mining (computed difficulty factor = 0.000244140625)
# This means current target is 4096 times the difficulty 1 target.
easy_factor = 0.000244140625  # equivalent to 1/4096
easy_target = int(difficulty_1_target / easy_factor)
print("Easy Mining Case:")
print("  Computed difficulty factor: ", easy_factor)
print("  Current target (int):       ", easy_target)
print("  Current target (hex):       ", hex(easy_target))
print("-" * 50)

# Case 2: Hard mining (computed difficulty factor = 4096)
# This means current target is 4096 times lower than the difficulty 1 target.
hard_factor = 4096
hard_target = int(difficulty_1_target / hard_factor)
print("Hard Mining Case:")
print("  Computed difficulty factor: ", hard_factor)
print("  Current target (int):       ", hard_target)
print("  Current target (hex):       ", hex(hard_target))
print("-" * 50)


Easy Mining Case:
  Computed difficulty factor:  0.000244140625
  Current target (int):        110426256551982323683968927107989468512300641233199776096820318605148160
  Current target (hex):        0xffff0000000000000000000000000000000000000000000000000000000
--------------------------------------------------
Hard Mining Case:
  Computed difficulty factor:  4096
  Current target (int):        6581917795657057981727655357598630697268285824847208028842229760
  Current target (hex):        0xffff0000000000000000000000000000000000000000000000000
--------------------------------------------------


Imagine the target as a finish line for a race—miners need to hit a hash value lower than this finish line to win. Here’s a simple breakdown:

Easy Mining Case (Difficulty Factor: 0.000244140625):

The computed factor is very small.

This means the finish line (target) is set very high—so many hashes can beat it.

In our example, the target number is huge (shown in both integer and hex forms), making it very easy to find a hash below it.

Hard Mining Case (Difficulty Factor: 4096):

The factor is much larger.

This means the finish line (target) is set much lower—only a few hashes can beat it.

The target in this case is lower, so miners must try many more combinations to find a valid hash.

In short:

A higher target (easy case) means more winning numbers are possible, so mining is easier.

A lower target (hard case) means fewer winning numbers are possible, so mining is harder.

Even though the numbers are enormous, the idea is just about where you set the finish line for winning—the higher it is, the easier it is to cross; the lower it is, the more challenging it becomes.








Lets see how increasing the difficulty works

In [3]:
# Helper function: Convert bits to full target (already explained)
def bits_to_target(bits):
    """
    Convert the compact representation of the target (bits) to the full target value.
    
    The 'bits' value is a 32-bit unsigned integer:
      - The most significant byte is the exponent.
      - The remaining 3 bytes are the coefficient.
    
    The target is computed as:
         target = coefficient * 2^(8*(exponent - 3))
    """
    exponent = bits >> 24             # Extract the exponent (top 8 bits)
    coefficient = bits & 0xffffff     # Extract the coefficient (lower 24 bits)
    target = coefficient * (1 << (8 * (exponent - 3)))
    return target

# Function: Convert a full target to the compact bits representation.
def target_to_bits(target):
    """
    Convert a full target integer into its compact representation (bits).
    """
    # Represent the target as a full 256-bit hexadecimal string.
    target_hex = '%064x' % target
    # Remove leading zeros.
    target_hex = target_hex.lstrip('0')
    # Ensure even length.
    if len(target_hex) % 2:
        target_hex = '0' + target_hex
    size = len(target_hex) // 2  # Number of bytes.
    
    # Extract the coefficient (first 3 bytes). If the target is smaller, shift it accordingly.
    if size <= 3:
        coefficient = int(target_hex, 16) << (8 * (3 - size))
    else:
        coefficient = int(target_hex[:6], 16)
    
    # Combine the size and coefficient into the 32-bit bits value.
    bits = (size << 24) | coefficient
    return bits

# Function: Compute current target from a given difficulty.
def difficulty_to_target(difficulty):
    """
    Compute the full target based on the given difficulty.
    
    In Bitcoin, the "difficulty 1 target" is defined by the bits value 0x1d00ffff.
    """
    # Define the difficulty 1 bits value (Bitcoin's genesis difficulty)
    difficulty_1_bits = 0x1d00ffff
    # Compute the full target for difficulty 1.
    difficulty_1_target = bits_to_target(difficulty_1_bits)
    # Compute current target based on the given difficulty.
    current_target = int(difficulty_1_target / difficulty)
    return current_target

# Function: Combine the above to get bits from difficulty.
def difficulty_to_bits(difficulty):
    current_target = difficulty_to_target(difficulty)
    return target_to_bits(current_target)

# Example usage: Compute bits and target for different difficulty levels.
# Iterate through different difficulty levels and print the target as hex and integer.
# List of difficulties to compare
difficulties = [1, 2, 4, 10]
results = []

for diff in difficulties:
    bits_val = difficulty_to_bits(diff)
    current_target = bits_to_target(bits_val)
    results.append((diff, bits_val, current_target))
    print(f"Difficulty: {diff}")
    print(f"  Bits:     {hex(bits_val)}")
    print(f"  Target (hex): {hex(current_target)}")
    print(f"  Target (int): {current_target}")
    print("-" * 50)



Difficulty: 1
  Bits:     0x1cffff00
  Target (hex): 0xffff0000000000000000000000000000000000000000000000000000
  Target (int): 26959535291011309493156476344723991336010898738574164086137773096960
--------------------------------------------------
Difficulty: 2
  Bits:     0x1c7fff80
  Target (hex): 0x7fff8000000000000000000000000000000000000000000000000000
  Target (int): 13479767645505654746578238172361995668005449369287082043068886548480
--------------------------------------------------
Difficulty: 4
  Bits:     0x1c3fffc0
  Target (hex): 0x3fffc000000000000000000000000000000000000000000000000000
  Target (int): 6739883822752827373289119086180997834002724684643541021534443274240
--------------------------------------------------
Difficulty: 10
  Bits:     0x1c199980
  Target (hex): 0x19998000000000000000000000000000000000000000000000000000
  Target (int): 2695953529101130949315647634472399133601089873857416408613777309696
--------------------------------------------------


In [4]:
# Find which difficulty has the lowest target (i.e. smallest integer value)
lowest = min(results, key=lambda x: x[2])
print(f"The lowest target is for Difficulty: {lowest[0]}, with a target integer value of {lowest[2]}")

The lowest target is for Difficulty: 10, with a target integer value of 2695953529101130949315647634472399133601089873857416408613777309696


This means that as difficulty increases, the current target becomes a smaller number (making it harder to find a hash below it).
For higher difficulty levels, the bits value changes to represent a lower target.